In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms,datasets
from torch import optim
import visdom

In [2]:
mnist_train = datasets.MNIST('./datasets/mnist_data',True,
    transform = transforms.Compose([
        transforms.ToTensor()
]),download = True)
mnist_train = DataLoader(mnist_train,batch_size = 32,shuffle = True)

mnist_test = datasets.MNIST('./datasets/mnist_data/',False,
    transform = transforms.Compose([
        transforms.ToTensor()
]),download = True)
mnist_test = DataLoader(mnist_test,batch_size = 32,shuffle = True)

In [3]:
# AE
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()

        # [b, 784] => [b, 20]
        self.encoder = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 20),
            nn.ReLU()
        )
        # [b, 20] => [b, 784]
        self.decoder = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, 784),
            nn.Sigmoid()
        )

    def forward(self, x):
        """
        :param x: [b, 1, 28, 28]
        :return:
        """
        batch_size = x.size(0)
        # flatten
        x = x.view(batch_size, 784)
        # encoder
        x = self.encoder(x)
        # decoder
        x = self.decoder(x)
        # reshape
        x = x.view(batch_size, 1, 28, 28)

        return x, None

In [4]:
%%time

device = torch.device('cuda')
model = AE().to(device)
criteon = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr = 1e-3)

viz = visdom.Visdom()

print(model)

for epoch in range(10):
    for batch_idx,(x,_) in enumerate(mnist_train):
        x = x.to(device)
        x_hat,_ = model(x)
        loss = criteon(x_hat,x)
        
        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'epoch : {epoch},loss : {loss.item()}')
    
    x,_ = iter(mnist_test).next()
    x = x.to(device)
    with torch.no_grad(): 
        x_hat,_ = model(x)
    viz.images(x,nrow = 8,win = 'x',opts = dict(title = 'x'))
    viz.images(x_hat,nrow = 8,win = 'x_hat',opts = dict(title = 'x_hat'))

Setting up a new session...


AE(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=20, bias=True)
    (5): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=20, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=784, bias=True)
    (5): Sigmoid()
  )
)
epoch : 0,loss : 0.024158088490366936
epoch : 1,loss : 0.01903233677148819
epoch : 2,loss : 0.01598658785223961
epoch : 3,loss : 0.014104633592069149
epoch : 4,loss : 0.013381687924265862
epoch : 5,loss : 0.014253390021622181
epoch : 6,loss : 0.014098677784204483
epoch : 7,loss : 0.012489549815654755
epoch : 8,loss : 0.011682047508656979
epoch : 9,loss : 0.011044216342270374
Wall time: 1min 56s
